## 1. Instalación y carga de librerías

In [10]:
!pip install nltk wordcloud
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from wordcloud import WordCloud
import re
from colorama import Fore
import requests

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached wordcloud-1.9.4-cp313-cp313-win_amd64.whl.metadata (3.5 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2025.8.29-cp313-cp313-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached contourpy-1.3.3-cp313-cp313-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached wordcloud-1.9.4-cp313-cp313-win_amd64.whl (300 kB)
Using cached regex-2025.8.29-cp313-cp313-win_amd64.whl (275 kB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached joblib-1


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package punkt to C:\Users\ROG BY
[nltk_data]     NG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\ROG BY
[nltk_data]     NG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\ROG BY
[nltk_data]     NG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL de Trustpilot (página de Airbnb)
base_url = "https://www.trustpilot.com/review/www.airbnb.com?page="

# Lista para guardar resultados
reviews = []

# Scrapear de la página 1 a la 65
for page in range(1, 66):
    print(f"📥 Bajando página {page} ...")
    
    url = base_url + str(page)
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(response.text, "html.parser")

    # Extraer reseñas (etiqueta <p>)
    comentarios = soup.find_all("p", {
        "class": "CDS_Typography_appearance-default__dd9b51 CDS_Typography_prettyStyle__dd9b51 CDS_Typography_body-l__dd9b51"
    })

    # Extraer fechas (etiqueta <time>)
    fechas = soup.find_all("time")

    for comentario, fecha in zip(comentarios, fechas):
        reviews.append({
            "fecha": fecha.text.strip(),
            "comentario": comentario.text.strip()
        })

    # Pausa para no ser baneado
    time.sleep(2)

# Convertir a DataFrame
df = pd.DataFrame(reviews)

# Guardar en CSV
df.to_csv("comentarios_airbnb_2025.csv", index=False, encoding="utf-8")

print("✅ Listo! Se guardaron", len(df), "comentarios en 'comentarios_airbnb_2025.csv'")




📥 Bajando página 1 ...
📥 Bajando página 2 ...
📥 Bajando página 3 ...
📥 Bajando página 4 ...
📥 Bajando página 5 ...
📥 Bajando página 6 ...
📥 Bajando página 7 ...
📥 Bajando página 8 ...
📥 Bajando página 9 ...
📥 Bajando página 10 ...
📥 Bajando página 11 ...
📥 Bajando página 12 ...
📥 Bajando página 13 ...
📥 Bajando página 14 ...
📥 Bajando página 15 ...
📥 Bajando página 16 ...
📥 Bajando página 17 ...
📥 Bajando página 18 ...
📥 Bajando página 19 ...
📥 Bajando página 20 ...
📥 Bajando página 21 ...
📥 Bajando página 22 ...
📥 Bajando página 23 ...
📥 Bajando página 24 ...
📥 Bajando página 25 ...
📥 Bajando página 26 ...
📥 Bajando página 27 ...
📥 Bajando página 28 ...
📥 Bajando página 29 ...
📥 Bajando página 30 ...
📥 Bajando página 31 ...
📥 Bajando página 32 ...
📥 Bajando página 33 ...
📥 Bajando página 34 ...
📥 Bajando página 35 ...
📥 Bajando página 36 ...
📥 Bajando página 37 ...
📥 Bajando página 38 ...
📥 Bajando página 39 ...
📥 Bajando página 40 ...
📥 Bajando página 41 ...
📥 Bajando página 42 ...
📥

,fecha,comentario
0,2 days ago,I stayed at Cascais Skyline Studio (Reservatio...
1,4 days ago,ZERO ZERO ZEROOOOO OR -1 STAR DESERVE. \nWe ar...
2,A day ago,A catastrophe. After being a superhost for man...
3,4 days ago,This company is a complete scam. They stole €6...
4,Just now,Our mission is to create a world where people ...
